# ORB from Andrew Aziz's paper on SSRN

https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4416622

Installing the required libraries

In [20]:
import pandas as pd
# import pandas_ta as ta
import time
import numpy as np
from datetime import timezone,timedelta,datetime
# from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import nest_asyncio
import yfinance as yf
import quantstats as qs
import mplfinance as mpf
import pickle
from  ib_insync import *


For the time being, we used data curated from various sources and filtered in a google drive csv file, this could be changed to be integrated from an external broker like IBKR for further use


In [2]:
df = pd.DataFrame()
columns = ['date', 'open', 'high', 'low', 'close', 'volume']
df = pd.DataFrame(columns=columns)

In [3]:
# Define the inputs
capital = 25000           # Initial account value
leverage = 4              # Leverage multiplier
commission_per_share = 0.0035       # Comission fees and slippage
account_risk=0.01         # Percentage of total account risked per trade
profit_multiplier=10      # Profit target multiplier from entry price
#first_column_index = df.index

In [8]:
import nest_asyncio
nest_asyncio.apply()
ib = IB()
ib.connect('127.0.0.1',7497,clientId=1)
contract = Stock('QQQ','SMART','USD')
def fetch_delayed_data():
    # Request delayed market data
    ib.reqMarketDataType(3)  # Use delayed data type (3) if real-time data is not available
    ticker = ib.reqMktData(contract, snapshot=True)
    ib.sleep(2)
    if ticker:
            data = {
                'date': pd.Timestamp.now(),
                'open': ticker.open,
                'high': ticker.high,
                'low': ticker.low,
                'close': ticker.close,
                'volume': ticker.volume,
            }
    return data
print(fetch_delayed_data())
ib.disconnect()

{'date': Timestamp('2024-09-12 15:38:29.465485'), 'open': 468.78, 'high': 474.04, 'low': 466.85, 'close': 468.62, 'volume': 316141.0}


In [14]:
import nest_asyncio
nest_asyncio.apply()
df = pd.DataFrame()
eod_price = 0
ib = IB()
ib.connect('127.0.0.1',7497,clientId=110)
contract = Stock('QQQ','SMART','USD')
def fetch_delayed_data():
    # Request delayed market data
    ib.reqMarketDataType(3)  # Use delayed data type (3) if real-time data is not available
    ticker = ib.reqMktData(contract, snapshot=True)
    ib.sleep(2)
    if ticker:
            data = {
                'date': pd.Timestamp.now(),
                'open': ticker.open,
                'high': ticker.high,
                'low': ticker.low,
                'close': ticker.close,
                'volume': ticker.volume,
            }
    return pd.DataFrame([data])
# print(fetch_delayed_data())
def is_end_of_day_data():
    # Current time in UTC
    current_time_utc = datetime.now(timezone.utc)

    # Convert UTC to New York timezone (EST or EDT depending on the time of the year)
    new_york_time = current_time_utc.astimezone(timezone(timedelta(hours=-4)))  # UTC-4 for Eastern Time during DST

    # Market close time is 4:00 PM EST or 16:00 hours
    market_close_time = new_york_time.replace(hour=16, minute=0, second=0, microsecond=0)

    # Check if the current time is close to the market close time (with a buffer of 5 minutes)
    return abs((new_york_time - market_close_time).total_seconds()) <= 300
try:
    while True:
        df = pd.concat([df,fetch_delayed_data()],ignore_index=True)
        print(df.tail())  # Print the most recent 5-minute bar
        time.sleep(61)  # Wait for 5 minutes
        if is_end_of_day_data():
            eod_price = 1
             
except KeyboardInterrupt:
    print("Data fetching stopped.")
finally:
    ib.disconnect()
df

                        date    open    high     low   close    volume
0 2024-09-12 15:43:53.121496  468.78  474.04  466.85  468.62  318940.0
                        date    open    high     low   close    volume
0 2024-09-12 15:43:53.121496  468.78  474.04  466.85  468.62  318940.0
1 2024-09-12 15:44:56.142695  468.78  474.04  466.85  468.62  319654.0
Data fetching stopped.


,date,open,high,low,close,volume
0,2024-09-12 15:43:53.121496,468.78,474.04,466.85,468.62,318940.0
1,2024-09-12 15:44:56.142695,468.78,474.04,466.85,468.62,319654.0


Creating a dataframe named **df1** which stores TQQQ data from 2020 to 2024 in 5 min time intervals, the Date was converted to a DateTimeIndex
> Certain Modifications to the data were required including:
1.   Converting the Time Zone to UTC
2.   Restricting the Data to the regular trading hours from 9:30 am to 4:00pm



In [23]:
with open('shared_dataframe.pkl','wb') as f:
    pickle.dump(df,f)

In [17]:
# df1 = pd.read_csv("/content/drive/MyDrive/2015-2024_TQQQ_5min_IBKR.csv")
# df1['date'] = pd.to_datetime(df1['date'])
df.set_index('date', inplace=True)
df.rename(columns={
            'Open': 'open',
            'High': 'high',
            'Low': 'low',
            'Close': 'close',
            'Volume': 'volume'
        }, inplace=True)
df.index = pd.to_datetime(df.index,utc=True)
df.index = df.index.tz_convert('UTC')
df.index = df.index.tz_localize(None)
df.index = df.index.floor('S')
df = df.between_time('09:30', '16:00')
df

/var/folders/_j/xlsbmg1j0x341kvd3qz81kgr0000gn/T/ipykernel_18015/1831483955.py:14: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df.index = df.index.floor('S')


,open,high,low,close,volume
date,,,,,
2024-09-12 15:43:53,468.78,474.04,466.85,468.62,318940.0
2024-09-12 15:44:56,468.78,474.04,466.85,468.62,319654.0


The MPL Finance Library was used to plot the data in a candle stick pattern, this would further enable us to visualize the various buy and sell markers and help us visualize where we buy and sell!

In [ ]:
def calculate_position_size(entry_price, stop_loss_price):
    risk_per_trade = capital * account_risk
    position_size = risk_per_trade / abs(entry_price - stop_loss_price)
    return position_size
# Example function to place a trade

In [ ]:
# Function to calculate ORB strategy
def place_trade(entry_price, stop_loss_price, target_price,type):
    position_size = calculate_position_size(entry_price, stop_loss_price)
    if type == 1:
        order = MarketOrder('BUY', position_size)
        trade = ib.placeOrder(contract, order)
        # Set stop loss and profit target
        stop_loss_order = StopOrder('SELL', position_size, stop_loss_price)
        profit_target_order = LimitOrder('SELL', position_size, target_price)
    elif type == -1:
        order = MarketOrder('SELL', position_size)
        trade = ib.placeOrder(contract, order)
        # Set stop loss and profit target
        stop_loss_order = StopOrder('BUY', position_size, stop_loss_price)
        profit_target_order = LimitOrder('BUY', position_size, target_price)
    # Place orders
    ib.placeOrder(contract, stop_loss_order)
    ib.placeOrder(contract, profit_target_order)
    
    return trade
entry_price = None
def apply_orb_strategy(data, capital, leverage, commission_per_share):
    # df_results = pd.DataFrame()
    # length = (len(data.groupby(data.index.date)))
    # df_results['Date'] = None
    # df_results['Date'] = df_results['Date'].reindex(range(length), fill_value=None)  # Pad with zeros if needed
    # df_results['ORB QQQ'] = ""
    # df_results['ORB QQQ'] = df_results['ORB QQQ'].reindex(range(length), fill_value=None)  # Pad with zeros if needed
    # df_results['Buy and Hold QQQ'] = ""
    # df_results['Buy and Hold QQQ'] = df_results['Buy and Hold QQQ'].reindex(range(length), fill_value=None)  # Pad with zeros if needed
    # buy_points = {'date':[],"price":[]}
    # sell_points = {'date':[],"price":[]}
    # results = []
    total_cap=capital
    count = 0
    initial_hold_quantity = capital/data['Close'][0]
    for date, day_data in data.groupby(data.index.date):
        day_data = day_data.reset_index()
        if day_data.empty or len(day_data) < 2:
            continue  # Skip if there's no data or not enough candles
        # Define the first 5-minute range
        opening_range = day_data.loc[0]
        open_price = opening_range['Open']
        high_price = opening_range['High']
        low_price = opening_range['Low']
        close_price = opening_range['Close']
        if close_price > open_price:# Bullish first candle
          entry_price = day_data.loc[1, 'Open']
          stop_loss = low_price
          direction = 1
        #   buy_points['date'].append(date)
        #   buy_points['price'].append(entry_price)

        elif close_price < open_price: # Bearish first candle
          entry_price = day_data.loc[1, 'Open']
          stop_loss = high_price
          direction = -1
        #   sell_points['date'].append(date)
        #   sell_points['price'].append(entry_price)
        else:
          direction = 0
          continue

        # Calculate risk per trade
        risk_per_trade = account_risk * capital
        # if entry_price == stop_loss:
        #   continue
        shares = risk_per_trade / abs(entry_price - stop_loss)
        shares = min(shares, capital * leverage / entry_price)  # Adjust for leverage

        profit_target = entry_price + direction * profit_multiplier * abs(entry_price - stop_loss)
        place_trade(entry_price, stop_loss, profit_target,direction)
        trade_commission = commission_per_share * shares
        for idx, row in day_data.iterrows():
            if idx <= 1:
                continue  # Skip the first two rows  ???????????????????????????? WHY SKIP TWO ROWS?

            current_price = row['Open']
            if direction == 1 and current_price >= profit_target:
                exit_price = profit_target
                # sell_points['date'].append(date)
                # sell_points['price'].append(exit_price)
                break
            elif direction == -1 and current_price <= profit_target:
                exit_price = profit_target
                # buy_points['date'].append(date)
                # buy_points['price'].append(exit_price)
                break
            elif direction == 1 and current_price <= stop_loss:
                exit_price = stop_loss
                # sell_points['date'].append(date)
                # sell_points['price'].append(exit_price)
                break
            elif direction == -1 and current_price >= stop_loss:
                exit_price = stop_loss
                # buy_points['date'].append(date)
                # buy_points['price'].append(exit_price)
                break
        else:
        exit_price = day_data.loc[len(day_data) - 1, 'Close']  # End of day exit
        trade_profit = direction * (exit_price - entry_price) * shares
        trade_profit -= trade_commission
        total_cap+=trade_profit
        
        count+=1
    return None
